## Load Movies data file

In [38]:
import pandas as pd
movies = pd.read_csv(r"C:\Users\91992\Downloads\movies.csv")#load movie list file
ratings = pd.read_csv(r"C:\Users\91992\Downloads\ml-25m\ratings.csv")#load rating file

print(movies.head())# Display the first few rows of the DF

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [39]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


# Preprocess movies name

In [32]:
import re

def clean_title(title):
    cleaned_title = re.sub("[^a-zA-Z0-9 ]", "", title) # Use regular expressions to remove any character that is not a letter, number, or space
    return cleaned_title

movies["clean_title"] = movies["title"].apply(clean_title)#apply it on movies file

print(movies)

       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
62418   209157                           We (2018)   
62419   209159           Window of the Soul (2001)   
62420   209163                    Bad Poems (2018)   
62421   209169                 A Girl Thing (2001)   
62422   209171      Women of Devil's Island (1962)   

                                            genres  \
0      Adventure|Animation|Children|Comedy|Fantasy   
1                       Adventure|Children|Fantasy   
2                                   Comedy|Romance   
3                             Comedy|Drama|Romance   
4                                           Comedy   
...                        

In [14]:
! pip install ipywidgets  # library to make search box

## Fuction to recommond movies based on rating of similar users

In [40]:
def find_similar_movies(movie_id):
    # identify users who rated the given movie more than 4
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    
    # movies that these similar users have also rated highly
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    
    # percentage of similar users who recommended each movie
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    
    # only include movies recommended by more than 10% of similar users
    similar_user_recs = similar_user_recs[similar_user_recs > 0.10]
    
    # to get recommendations from all users who have rated these movies highly
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    # combine the data from similar users and all users
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    #calculate a recommendation score
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    
    #sort by score in descending order
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    
    # Return the top 10 recommendations, including movie title and genres
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]


In [42]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()